In [ ]:
import os
from tqdm import tqdm
import torchaudio.functional as F
import pandas as pd
import torchaudio
from other.data.audio_utils import AudioWorker

In [ ]:
# Be sure that data is resampled
data_dir = r'accent-dataset\common_accent'
labels_path = r'datasets\16000_silero_labels.csv'

In [ ]:
pitch_values = [-2, -1, 1, 2]

In [ ]:
labels = pd.read_csv(labels_path).dropna()
final_dicts = []

_tqdm = tqdm(labels.iterrows(), total=len(labels))
for idx, row in _tqdm:
    name, ext = row.filename.split('.')
    audio_file_path = os.path.join(data_dir, row.filename)

    if not os.path.exists(audio_file_path):
        continue

    aw = AudioWorker(audio_file_path, os.path.basename(audio_file_path))
    aw.load().leave_one_channel()
    final_dicts.append({'filename': row.filename, 'labels': row.labels})

    for pv in pitch_values:
        save_name = name + f'_{pv}' + '.' + ext
        save_path = os.path.join(data_dir, save_name)

        if os.path.exists(save_path):
            final_dicts.append({'filename': save_name, 'labels': row.labels})
            continue

        wave = F.pitch_shift(aw.wave.to('cuda'), sample_rate=aw.rate, n_steps=pv)

        torchaudio.save(save_path, wave.detach().cpu(), aw.rate)
        # print(save_path)

        final_dicts.append({'filename': save_name, 'labels': row.labels})

    # row.filename = name + ''.join(['{_' + str(pv) + '}' for pv in pitch_values]) + '.' + ext

new_labels = pd.DataFrame(final_dicts)
new_labels.to_csv(os.path.basename(labels_path).split('.')[0] + "_pitched.csv", index=False)